### quantify site specific greenery 

source data: berlin geoportal: shapefile with polygones and concrete green volume in m3 per m2

output variables:

`GVI_x`: Green Volume Index as weighted average for radius size x = [25,50,75,100,200]

`LAI`: factor of leaf area index/ change of biomass over time



In [1]:
# 1 load data and select features
import geopandas

sites = geopandas.read_file('data/monitoring_station/monitoring_station.shp')[['id','stattyp', 'geometry']]
greenery = geopandas.read_file('data/green_volume/green_volume.shp')[['schl5', 'vegvol2020', 'geometry']]

# 1.2 check reference system - no adjustment needed 
print(f'Coordinate Reference System monitoring sites: {sites.crs}')
print(f'Coordinate Reference System green volume: {greenery.crs}')

Coordinate Reference System monitoring sites: EPSG:25833
Coordinate Reference System green volume: EPSG:25833


In [2]:
# calculate weight mean vegetation volume index (VVI) for different radius sizes and different sites and append to df
import geopandas 
import matplotlib.pyplot as plt

def calc_GVI(coordinate, radius, id, plot = False):
    
    # intersect radius with map of greenery
    buffer  = geopandas.GeoDataFrame(geometry= [coordinate.buffer(radius)], crs=greenery.crs) # create polygon with buffer region around monitoring point
    in_buffer_one = geopandas.overlay(greenery, buffer, how='intersection') # intersect block areas (with vegetation volume) with radius polygon to reduce block areas inside the radius 
    
    # calculate weighted mean of vegetation volume inside each circle
    in_buffer_one['area'] = in_buffer_one['geometry'].area # area for each block inside circle  
    in_buffer_one['factor_greenery'] = in_buffer_one['area'] / in_buffer_one['area'].sum() # weight= relative proportion of subarea from total area inside circle (sum == 1)
    in_buffer_one['weighted_greenery'] = (in_buffer_one['factor_greenery']) * in_buffer_one['vegvol2020'] # weight
    current_GVI = round(in_buffer_one['weighted_greenery'].sum(),3) # absolute VVI as sum of weighted area index

  
    # plot buffer zones 
    if plot == True: 
        ax = in_buffer_one.plot(column='vegvol2020', cmap='Greens', legend=True, edgecolor='black', vmin= 0, vmax = 10,)
        ax.set_title(f'{id}: {current_GVI}')
        plt.savefig(f"data/green_volume/output/buffer_{(id.lower()).replace(' ','')}_{radius}.png", dpi = 300)
        plt.show()
    
    return current_GVI


plot_greenery = False # adjust if 

# apply for all radius and all sites
for radius in [25,50,75,100,200]:
    sites[f'GVI_{radius}'] = sites.apply(lambda row: calc_GVI(coordinate=row['geometry'], radius=radius, id=row['id'], plot = plot_greenery), axis=1)
    
sites.to_file('data/green_volume/buffer_values/buffer_values.shp')
sites.head(5)

,id,stattyp,geometry,GVI_25,GVI_50,GVI_75,GVI_100,GVI_200
0,MC 042,Wohngebiet,POINT (393459.020 5816635.250),4.050,3.672,3.617,3.570,3.171
1,MC 124,Verkehr,POINT (390406.146 5810991.699),2.521,2.224,2.426,2.558,3.589
2,MC 143,Verkehr,POINT (394135.247 5814178.610),1.300,1.446,1.720,1.793,1.660
3,MC 171,Wohngebiet,POINT (392699.560 5819341.461),0.546,0.499,0.481,0.492,0.772
4,MC 174,Verkehr,POINT (396182.715 5819313.198),1.012,1.865,2.272,2.403,2.275


In [3]:
# plot 
import matplotlib.pyplot as plt 

sites['GVI_25'].plot('x')
sites['GVI_50'].plot('x')
sites['GVI_75'].plot('x')
sites['GVI_100'].plot('x')
sites['GVI_200'].plot('x')
plt.title("Vegetation Volume Index for different Sites and Radius")
plt.xlabel("Different Monetoring Sites")
plt.ylim(ymin=0.2, ymax=20)
plt.legend()

TypeError: `Series.plot()` should not be called with positional arguments, only keyword arguments. The order of positional arguments will change in the future. Use `Series.plot(kind='x')` instead of `Series.plot('x',)`.

In [ ]:
# plot greenery 
import matplotlib.pyplot as plt 

for n in range(len(sites['id'])):
    name = sites['id'][n]
    
    radius = [25,50,75,100,200,300]
    coordinate = sites['geometry'][n]


    fig, axs = plt.subplots((len(radius)//2), 2, figsize=(12, 8), dpi=150)
    fig.suptitle(f'Vegetation Volume Index inside radius at {name}:', fontsize =15)


    for n in range(len(radius)):
        
        # intersect 
        buffer  = geopandas.GeoDataFrame(geometry= [coordinate.buffer(radius[n])], crs=greenery.crs)
        in_buffer_one = geopandas.overlay(greenery, buffer, how='intersection')
        
        # calc weighted mean VVI
        in_buffer_one['area'] = in_buffer_one['geometry'].area 
        in_buffer_one['factor_greenery'] = in_buffer_one['area'] / in_buffer_one['area'].sum()
        in_buffer_one['weighted_greenery'] = (in_buffer_one['factor_greenery']) * in_buffer_one['vegvol2020']
        current_GVI = round(in_buffer_one['weighted_greenery'].sum(),3)

        # plot different VVI over radius
        row = n//2
        column = 0 if (n+1)%2 != 0 else 1
        
        ax = in_buffer_one.plot(ax =axs[row, column] , 
                                column='vegvol2020', 
                                cmap='Greens', vmin= 0, vmax = 9,
                                legend=True, edgecolor='black')
        axs[row, column].set_title(f'VVI for {radius[n]}m radius: {current_GVI}', {'fontsize':11})
        axs[row, column].set_xticklabels([])
        axs[row, column].set_yticklabels([])
        plt.tight_layout()
           
    #plt.savefig(f"data/green_volume/output/compare_{name}.png", dpi = 450)
    plt.show()


### most informative radius area

In [61]:
#  distribution between sites
import pandas as pd
sites_analysis = sites.drop(['geometry', 'stattyp'], axis= 1)
sites_analysis['id'] = sites_analysis['id'].apply(lambda x: x.lower().replace(' ',''))
sites_analysis = sites_analysis.set_index('id')
sites_analysis = sites.drop(['geometry', 'stattyp'], axis= 1)
sites_analysis['id'] = sites_analysis['id'].apply(lambda x: x.lower().replace(' ',''))
sites_analysis = sites_analysis.set_index('id')
sites_difference = pd.DataFrame((sites_analysis.std(axis= 1) / sites_analysis.mean(axis = 1))*100 , columns=['rstd'])
sites_difference['min_to_max'] = sites_analysis.max(axis= 1) - sites_analysis.min(axis= 1) 
sites_difference['std'] = sites_analysis.std(axis =1)

pd.concat([sites_analysis, sites_difference], axis =1)
# RESULT: low variance between different sites

,GVI_25,GVI_50,GVI_75,GVI_100,GVI_200,rstd,min_to_max,std
id,,,,,,,,
mc042,4.050,3.672,3.617,3.570,3.171,8.653300,0.879,0.312903
mc124,2.521,2.224,2.426,2.558,3.589,20.021050,1.365,0.533281
mc143,1.300,1.446,1.720,1.793,1.660,12.926325,0.493,0.204727
mc171,0.546,0.499,0.481,0.492,0.772,21.894992,0.291,0.122174
mc174,1.012,1.865,2.272,2.403,2.275,29.010668,1.391,0.570176
mc190,0.292,0.327,0.338,0.344,0.472,19.359796,0.180,0.068650
mc221,0.820,0.962,1.184,1.244,1.749,29.792809,0.929,0.355071
mc145,16.366,16.356,16.185,16.155,17.965,4.613544,1.810,0.766097
mc077,18.936,17.993,16.067,14.902,14.145,12.343344,4.791,2.025370


In [116]:
# load emission data
import pandas as pd 
from sklearn.feature_selection import mutual_info_regression


no2 = pd.read_csv('data/datasets/df_processed.csv')[['id', 'Stickstoffdioxid']].groupby('id').mean().rename({'Stickstoffdioxid': 'NO2'}, axis =1)
no2_vvi = pd.concat([no2, sites_analysis], axis= 1).sort_values('NO2', axis=0)

results = {'correlation' : [],
           'mutual_info' : [],}

for radius in list(sites_analysis.columns):
    results['correlation'].append(round(no2_vvi['NO2'].corr(no2_vvi[radius], method='pearson'),4))
    results['mutual_info'].append(round(mutual_info_regression(no2_vvi.filter(['NO2']), no2_vvi[radius])[0],4))


results = pd.DataFrame(results).transpose()
results.columns = list(sites_analysis.columns)
results

,GVI_25,GVI_50,GVI_75,GVI_100,GVI_200
correlation,-0.6605,-0.6639,-0.6576,-0.6577,-0.6600
mutual_info,0.1503,0.1012,0.1058,0.1128,0.1101


## leaf area index
1. use grid files to extract mean leaf area index for regions around berlin (8 day resolution)
2. load data in from different years in data frame, normalize and build mean
3. extrapolate leaf are index over every timestep

In [2]:
import rioxarray

# define function to process grid of leaf are index .tif 

def mean_in_grid(file_path, bbox):
    """
    crop grid to relevant region and return mean in value (LAI) of grid 
    bbox: buffer to filter the data, in the format (min_lon, min_lat, max_lon, max_lat)
    return: mean
    """
    
    data = rioxarray.open_rasterio(file_path) # Load the data with rioxarray
   
    # Define the bounding box for Berlin and filter
    min_lon, min_lat, max_lon, max_lat = bbox 
    data_clipped = data.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat) # filter
    first_band = data_clipped.sel(band=1)

    # Convert the clipped data to a DataFrame
    df = first_band.to_series().reset_index().rename(columns={0: "Value"})
    df = df[df['Value'] != 255] # drop rows with fill (na) value 255


    return round(df['Value'].mean() * 0.1,3)  # return only mean of LAI Value

# generate time steps as index to load .tif data (8 day resolution)
index_day = []

for value in [ x for x in range(1,362,8)]:
    if len(str(value)) == 1:
        index_day.append('00' +str(value))  
    elif len(str(value)) == 2:
        index_day.append('0' +str(value))  
    else:
        index_day.append(str(value))  


# load data for each time step
lai_mean_timesteps = {}

for n in range(len(index_day)):
    current_year = []
    for year in ['2017', '2018', '2019', '2020', '2021', '2022']:

        path = f"data/green_volume/lai/HiQ_LAI_WGS84_5km_8day_{year + index_day[n]}.tif" # define current path to .tif file
        try:
            current_year.append(mean_in_grid(path, bbox= (12.4, 51.5, 14.4, 53.5))) # extract mean lai for this file
        except:
            current_year.append(lai_mean_timesteps.get(index_day[n-1])[-1])
            print(f'timestep: {index_day[n]} not found!') 

    lai_mean_timesteps[index_day[n]] = current_year


timestep: 289 not found!


In [3]:
# transform leaf area index to 
import pandas as pd

lai_df = pd.DataFrame(lai_mean_timesteps).transpose().rename(columns={0:'2017', 1: '2018', 2:'2019', 3: '2020', 4:'2021', 5: '2022'})

for year in ['2017', '2018', '2019', '2020', '2021', '2022']:
    lai_df[year] = lai_df[year].apply(lambda x: x/ lai_df[year].max())
    
lai_df['lai_factor'] = (lai_df['2017'] + lai_df['2018']+ lai_df['2019'] + lai_df['2020']+ lai_df['2021'] + lai_df['2022']) / 6
lai_df


,2017,2018,2019,2020,2021,2022,lai_factor
001,0.212213,0.346486,0.319563,0.535135,0.154615,0.525335,0.348891
009,0.264983,0.441563,0.451195,0.627928,0.260000,0.671606,0.452879
017,0.298153,0.360767,0.520049,0.736486,0.395000,0.743786,0.509040
025,0.336977,0.376174,0.434184,0.440541,0.358077,0.480402,0.404392
033,0.262721,0.345735,0.366950,0.339640,0.302692,0.436424,0.342360
041,0.240106,0.321684,0.350344,0.334234,0.271154,0.401530,0.319842
049,0.238598,0.311537,0.326853,0.310811,0.314615,0.423518,0.320989
057,0.224651,0.291244,0.317942,0.383333,0.306923,0.456501,0.330099
065,0.263852,0.291620,0.360470,0.440090,0.328846,0.442161,0.354506
073,0.309461,0.272454,0.366545,0.476577,0.382692,0.451721,0.376575


In [4]:
# combine factor with dates
import pandas as pd
import datetime


def assin_lai(date):
    # date = e.g. 2023010100
    date = str(date)

    # 1. extract week nr
    day_nr = str(datetime.date(int(date[:4]), int(date[4:6]), int(date[6:8])).timetuple().tm_yday)
    if len(day_nr) == 1:
        day_nr = "00" + day_nr
    elif len(day_nr) == 2:
        day_nr = "0" + day_nr

    # 2. return leaf are index factor based on week nr.
    try:
        return lai_df[lai_df.index == day_nr]["lai_factor"].iloc[0]

    except:
        return None


timesteps = pd.read_csv('data/weather/df_weather_cleaned.csv').filter(['MESS_DATUM'])
timesteps['lai_factor'] = timesteps['MESS_DATUM'].apply(lambda x: assin_lai(x))
timesteps = timesteps.ffill()
timesteps.to_csv("data/datasets/lai_factor.csv")
timesteps


,MESS_DATUM,lai_factor
0,2023010100,0.348891
1,2023010101,0.348891
2,2023010102,0.348891
3,2023010103,0.348891
4,2023010104,0.348891
...,...,...
8755,2023123119,0.291876
8756,2023123120,0.291876
8757,2023123121,0.291876
8758,2023123122,0.291876
